In [1]:
import torch 
import os
os.chdir('02514-Deep-Learning-In-Computer-Vision/gan')

In [2]:
import pickle

with open('ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cpu()  # torch.nn.Module

In [7]:
z = torch.randn([1, G.z_dim]).cuda()    # latent codes
c = None     
w = G.mapping(z, c, truncation_psi=0.5, truncation_cutoff=8)
img = G.synthesis(w, noise_mode='const', force_fp32=True)